# Span Batch Analysis - Data Preparation

## Initialization

Before start, Fill in [secret.yml](span-batch-analysis/secret.yaml). Refer to [secret_example.yml](span-batch-analysis/secret_example.yaml). This is for specifying RPC endpoints.

Also check config located at [config](span-batch-analysis/config/).

In [1]:
import os

Build binaries.

In [2]:
%%bash
cd cmd/batch_decoder && go build . && cd ../../
cd cmd/span_batch_tester && go build . && cd ../../

In [3]:
%%capture
%env BATCH_DECODER=cmd/batch_decoder/batch_decoder
%env SPAN_BATCH_TESTER=cmd/span_batch_tester/span_batch_tester

Clone superchain config.

In [4]:
%%bash
git clone https://github.com/ethereum-optimism/superchain-registry
cd superchain-registry && git checkout 5d3ca1966435827ac8c4216bddcb541ec64c1d6c && cd --

fatal: destination path 'superchain-registry' already exists and is not an empty directory.
HEAD is now at 5d3ca19 superchain/extra: include minified genesis state data (#2)


Select your chain.

In [5]:
%%capture
%env L1_CHAIN_NAME=mainnet
%env L2_CHAIN_NAME=zora

# available networks
#      L1 - L2 list
# mainnet - base, op, pgn, zora
# goerli  - base, op, zora
# sepolia - op, pgn

In [6]:
l1_chain_name = os.environ["L1_CHAIN_NAME"]
l2_chain_name = os.environ["L2_CHAIN_NAME"]

start_number = !yq eval ".start_l1_number" "span-batch-analysis/config/{l1_chain_name}/{l2_chain_name}.yaml"
start_number_value = int(start_number[0])
end_number = !yq eval ".end_l1_number" "span-batch-analysis/config/{l1_chain_name}/{l2_chain_name}.yaml"
end_number_value = int(end_number[0])
batch_tx_sender = !yq eval ".batch_tx_sender" "span-batch-analysis/config/{l1_chain_name}/{l2_chain_name}.yaml"
batch_tx_sender_value = batch_tx_sender[0]
l2_genesis_timestamp = !yq eval ".genesis.l2_time" "superchain-registry/superchain/configs/{l1_chain_name}/{l2_chain_name}.yaml"
l2_genesis_timestamp_value = int(l2_genesis_timestamp[0])
l2_chain_id = !yq eval ".chain_id" "superchain-registry/superchain/configs/{l1_chain_name}/{l2_chain_name}.yaml"
l2_chain_id_value = int(l2_chain_id[0])
inbox_addr = !yq eval ".batch_inbox_addr" "superchain-registry/superchain/configs/{l1_chain_name}/{l2_chain_name}.yaml"
inbox_addr_value = inbox_addr[0]

acc_start_number = !yq eval ".acc_l2_start_number" "span-batch-analysis/config/{l1_chain_name}/{l2_chain_name}.yaml"
acc_start_number_value = int(acc_start_number[0])
acc_end_number = !yq eval ".acc_l2_end_number" "span-batch-analysis/config/{l1_chain_name}/{l2_chain_name}.yaml"
acc_end_number_value = int(acc_end_number[0])

Read analyis config.

In [7]:
%env L1_START=$start_number_value
%env L1_END=$end_number_value
%env BATCH_TX_SENDER=$batch_tx_sender_value
%env L2_GENESIS_TIMESTAMP=$l2_genesis_timestamp_value
%env L2_CHAIN_ID=$l2_chain_id_value
%env INBOX=$inbox_addr_value

%env ACC_L2_START=$acc_start_number_value
%env ACC_L2_END=$acc_end_number_value


env: L1_START=18000000
env: L1_END=18020000
env: BATCH_TX_SENDER=0x625726c858dBF78c0125436C943Bf4b4bE9d9033
env: L2_GENESIS_TIMESTAMP=1686693839
env: L2_CHAIN_ID=7777777
env: INBOX=0x6F54Ca6F6EdE96662024Ffd61BFd18f3f4e34DFf
env: ACC_L2_START=3348000
env: ACC_L2_END=3349000


Get L1 RPC from secret.

In [8]:
key_name = f"L1_{l1_chain_name}_RPC".upper()
L1_RPC = !yq eval ".{key_name}" span-batch-analysis/secret.yaml
L1_RPC_ENDPOINT = L1_RPC[0]

key_name = f"L2_{l2_chain_name}_{l1_chain_name}_RPC".upper()
L2_RPC = !yq eval ".{key_name}" span-batch-analysis/secret.yaml
L2_RPC_ENDPOINT = L2_RPC[0]

In [9]:
%%capture
%env L1_RPC=$L1_RPC_ENDPOINT
%env L2_RPC=$L2_RPC_ENDPOINT

## Data Collection

All data will be stored at `./artifacts/{L1_CHAIN_NAME}/{L2_CHAIN_NAME}`

In [10]:
l1_chain_name = os.environ["L1_CHAIN_NAME"]
l2_chain_name = os.environ["L2_CHAIN_NAME"]

%env DB_PREFIX=./artifacts/$l1_chain_name/$l2_chain_name

env: DB_PREFIX=./artifacts/mainnet/zora


batch decoder `fetch`: Read channel frames from L1 by reading batch transactions.

In [11]:
!$BATCH_DECODER fetch --l1=$L1_RPC --start=$L1_START --end=$L1_END --inbox=$INBOX --out=$DB_PREFIX/transactions_cache --sender=$BATCH_TX_SENDER --concurrent-requests=25 | tail -10

Fetched block:  18019995
Fetched block:  18019999
Fetched block:  18019998
Fetched block:  18019988
Fetched block:  18019994
Fetched block:  18019997
Fetched block:  18019993
Fetched batches in range [18000000,18020000). Found 408 valid & 0 invalid batches
Fetch Config: Chain ID: 1. Inbox Address: 0x6F54Ca6F6EdE96662024Ffd61BFd18f3f4e34DFf. Valid Senders: map[0x625726c858dBF78c0125436C943Bf4b4bE9d9033:{}].
Wrote transactions with batches to ./artifacts/mainnet/zora/transactions_cache


batch decoder `reassemble`: Reassemble channel frames to channels. 

In [12]:
!$BATCH_DECODER reassemble --inbox=$INBOX --in=$DB_PREFIX/transactions_cache --out=$DB_PREFIX/channel_cache | tail -10

span batch tester `convert`: Convert channels with singular batches to span batches. Try multiple tx encoding types.

In [13]:
!TX_ENCODE_TYPE=0  && $SPAN_BATCH_TESTER convert --in=$DB_PREFIX/channel_cache --out=$DB_PREFIX/span_batch_cache_v$TX_ENCODE_TYPE --l2=$L2_RPC --genesis-timestamp=$L2_GENESIS_TIMESTAMP --chain-id=$L2_CHAIN_ID --tx-type=$TX_ENCODE_TYPE | tail -10

[399/408] Channel ID: f83fc6b32c38bcbdd3ae9c6f063b8801, L2StartNum: 3294052, L2EndNum: 3294351, L2BlockCnt, 300
[400/408] Channel ID: f869d64673aa2f4e04aef4edae39656c, L2StartNum: 3215067, L2EndNum: 3215361, L2BlockCnt, 295
[401/408] Channel ID: f8e958456d48e3f6a7419c9586ecdf23, L2StartNum: 3247154, L2EndNum: 3247453, L2BlockCnt, 300
[402/408] Channel ID: f9e7149e3f12f9b22f13d5be43afaecc, L2StartNum: 3245084, L2EndNum: 3245378, L2BlockCnt, 295
[403/408] Channel ID: fa9a9b2c115638768e19c086c1f19835, L2StartNum: 3187852, L2EndNum: 3188143, L2BlockCnt, 292
[404/408] Channel ID: fb4bf2a168cb421815dbe98e5f315acf, L2StartNum: 3235844, L2EndNum: 3236138, L2BlockCnt, 295
[405/408] Channel ID: fdd5ea1e64138e9746c6ee85def32272, L2StartNum: 3228122, L2EndNum: 3228416, L2BlockCnt, 295
[406/408] Channel ID: fe8aebaa27f7b356db104b6d8fcf710e, L2StartNum: 3229917, L2EndNum: 3230208, L2BlockCnt, 292
[407/408] Channel ID: fee1bb6ecbcc15d19026d6c0c1d02a8f, L2StartNum: 3292539, L2EndNum: 3292838, L2BlockC

In [14]:
!TX_ENCODE_TYPE=1  && $SPAN_BATCH_TESTER convert --in=$DB_PREFIX/channel_cache --out=$DB_PREFIX/span_batch_cache_v$TX_ENCODE_TYPE --l2=$L2_RPC --genesis-timestamp=$L2_GENESIS_TIMESTAMP --chain-id=$L2_CHAIN_ID --tx-type=$TX_ENCODE_TYPE | tail -10

[399/408] Channel ID: f83fc6b32c38bcbdd3ae9c6f063b8801, L2StartNum: 3294052, L2EndNum: 3294351, L2BlockCnt, 300
[400/408] Channel ID: f869d64673aa2f4e04aef4edae39656c, L2StartNum: 3215067, L2EndNum: 3215361, L2BlockCnt, 295
[401/408] Channel ID: f8e958456d48e3f6a7419c9586ecdf23, L2StartNum: 3247154, L2EndNum: 3247453, L2BlockCnt, 300
[402/408] Channel ID: f9e7149e3f12f9b22f13d5be43afaecc, L2StartNum: 3245084, L2EndNum: 3245378, L2BlockCnt, 295
[403/408] Channel ID: fa9a9b2c115638768e19c086c1f19835, L2StartNum: 3187852, L2EndNum: 3188143, L2BlockCnt, 292
[404/408] Channel ID: fb4bf2a168cb421815dbe98e5f315acf, L2StartNum: 3235844, L2EndNum: 3236138, L2BlockCnt, 295
[405/408] Channel ID: fdd5ea1e64138e9746c6ee85def32272, L2StartNum: 3228122, L2EndNum: 3228416, L2BlockCnt, 295
[406/408] Channel ID: fe8aebaa27f7b356db104b6d8fcf710e, L2StartNum: 3229917, L2EndNum: 3230208, L2BlockCnt, 292
[407/408] Channel ID: fee1bb6ecbcc15d19026d6c0c1d02a8f, L2StartNum: 3292539, L2EndNum: 3292838, L2BlockC

In [15]:
!TX_ENCODE_TYPE=2  && $SPAN_BATCH_TESTER convert --in=$DB_PREFIX/channel_cache --out=$DB_PREFIX/span_batch_cache_v$TX_ENCODE_TYPE --l2=$L2_RPC --genesis-timestamp=$L2_GENESIS_TIMESTAMP --chain-id=$L2_CHAIN_ID --tx-type=$TX_ENCODE_TYPE | tail -10

[399/408] Channel ID: f83fc6b32c38bcbdd3ae9c6f063b8801, L2StartNum: 3294052, L2EndNum: 3294351, L2BlockCnt, 300
[400/408] Channel ID: f869d64673aa2f4e04aef4edae39656c, L2StartNum: 3215067, L2EndNum: 3215361, L2BlockCnt, 295
[401/408] Channel ID: f8e958456d48e3f6a7419c9586ecdf23, L2StartNum: 3247154, L2EndNum: 3247453, L2BlockCnt, 300
[402/408] Channel ID: f9e7149e3f12f9b22f13d5be43afaecc, L2StartNum: 3245084, L2EndNum: 3245378, L2BlockCnt, 295
[403/408] Channel ID: fa9a9b2c115638768e19c086c1f19835, L2StartNum: 3187852, L2EndNum: 3188143, L2BlockCnt, 292
[404/408] Channel ID: fb4bf2a168cb421815dbe98e5f315acf, L2StartNum: 3235844, L2EndNum: 3236138, L2BlockCnt, 295
[405/408] Channel ID: fdd5ea1e64138e9746c6ee85def32272, L2StartNum: 3228122, L2EndNum: 3228416, L2BlockCnt, 295
[406/408] Channel ID: fe8aebaa27f7b356db104b6d8fcf710e, L2StartNum: 3229917, L2EndNum: 3230208, L2BlockCnt, 292
[407/408] Channel ID: fee1bb6ecbcc15d19026d6c0c1d02a8f, L2StartNum: 3292539, L2EndNum: 3292838, L2BlockC

span batch tester `analyze`: Analyze channels with singluar batches by comparing with corresponding span batches.

In [16]:
!TX_ENCODE_TYPE=0 && $SPAN_BATCH_TESTER analyze --in-channel=$DB_PREFIX/channel_cache --in-span-batch=$DB_PREFIX/span_batch_cache_v$TX_ENCODE_TYPE --out=$DB_PREFIX/result_v$TX_ENCODE_TYPE --chain-id=$L2_CHAIN_ID --tx-type=$TX_ENCODE_TYPE | tail -10

[399/408] Channel ID: f83fc6b32c38bcbdd3ae9c6f063b8801, CompressedReductionPercent: 25.258686 %
[400/408] Channel ID: f869d64673aa2f4e04aef4edae39656c, CompressedReductionPercent: 25.056463 %
[401/408] Channel ID: f8e958456d48e3f6a7419c9586ecdf23, CompressedReductionPercent: 31.772283 %
[402/408] Channel ID: f9e7149e3f12f9b22f13d5be43afaecc, CompressedReductionPercent: 36.931331 %
[403/408] Channel ID: fa9a9b2c115638768e19c086c1f19835, CompressedReductionPercent: 24.811493 %
[404/408] Channel ID: fb4bf2a168cb421815dbe98e5f315acf, CompressedReductionPercent: 25.873518 %
[405/408] Channel ID: fdd5ea1e64138e9746c6ee85def32272, CompressedReductionPercent: 25.262607 %
[406/408] Channel ID: fe8aebaa27f7b356db104b6d8fcf710e, CompressedReductionPercent: 24.228526 %
[407/408] Channel ID: fee1bb6ecbcc15d19026d6c0c1d02a8f, CompressedReductionPercent: 22.106232 %
[408/408] Channel ID: ff16122882d3cc26a3111f32353f620f, CompressedReductionPercent: 28.473456 %


In [17]:
!TX_ENCODE_TYPE=1 && $SPAN_BATCH_TESTER analyze --in-channel=$DB_PREFIX/channel_cache --in-span-batch=$DB_PREFIX/span_batch_cache_v$TX_ENCODE_TYPE --out=$DB_PREFIX/result_v$TX_ENCODE_TYPE --chain-id=$L2_CHAIN_ID --tx-type=$TX_ENCODE_TYPE | tail -10

[399/408] Channel ID: f83fc6b32c38bcbdd3ae9c6f063b8801, CompressedReductionPercent: 22.636929 %
[400/408] Channel ID: f869d64673aa2f4e04aef4edae39656c, CompressedReductionPercent: 21.925134 %
[401/408] Channel ID: f8e958456d48e3f6a7419c9586ecdf23, CompressedReductionPercent: 29.526119 %
[402/408] Channel ID: f9e7149e3f12f9b22f13d5be43afaecc, CompressedReductionPercent: 34.733825 %
[403/408] Channel ID: fa9a9b2c115638768e19c086c1f19835, CompressedReductionPercent: 22.145029 %
[404/408] Channel ID: fb4bf2a168cb421815dbe98e5f315acf, CompressedReductionPercent: 22.568315 %
[405/408] Channel ID: fdd5ea1e64138e9746c6ee85def32272, CompressedReductionPercent: 22.191346 %
[406/408] Channel ID: fe8aebaa27f7b356db104b6d8fcf710e, CompressedReductionPercent: 20.785123 %
[407/408] Channel ID: fee1bb6ecbcc15d19026d6c0c1d02a8f, CompressedReductionPercent: 19.961872 %
[408/408] Channel ID: ff16122882d3cc26a3111f32353f620f, CompressedReductionPercent: 25.239594 %


In [18]:
!TX_ENCODE_TYPE=2 && $SPAN_BATCH_TESTER analyze --in-channel=$DB_PREFIX/channel_cache --in-span-batch=$DB_PREFIX/span_batch_cache_v$TX_ENCODE_TYPE --out=$DB_PREFIX/result_v$TX_ENCODE_TYPE --chain-id=$L2_CHAIN_ID --tx-type=$TX_ENCODE_TYPE | tail -10

[399/408] Channel ID: f83fc6b32c38bcbdd3ae9c6f063b8801, CompressedReductionPercent: 26.734942 %
[400/408] Channel ID: f869d64673aa2f4e04aef4edae39656c, CompressedReductionPercent: 26.742339 %
[401/408] Channel ID: f8e958456d48e3f6a7419c9586ecdf23, CompressedReductionPercent: 32.308977 %
[402/408] Channel ID: f9e7149e3f12f9b22f13d5be43afaecc, CompressedReductionPercent: 38.055807 %
[403/408] Channel ID: fa9a9b2c115638768e19c086c1f19835, CompressedReductionPercent: 27.015810 %
[404/408] Channel ID: fb4bf2a168cb421815dbe98e5f315acf, CompressedReductionPercent: 27.777195 %
[405/408] Channel ID: fdd5ea1e64138e9746c6ee85def32272, CompressedReductionPercent: 27.005558 %
[406/408] Channel ID: fe8aebaa27f7b356db104b6d8fcf710e, CompressedReductionPercent: 25.966162 %
[407/408] Channel ID: fee1bb6ecbcc15d19026d6c0c1d02a8f, CompressedReductionPercent: 23.956087 %
[408/408] Channel ID: ff16122882d3cc26a3111f32353f620f, CompressedReductionPercent: 30.094783 %


span batch tester `fetch`: Fetch consecutive singular batches.

In [19]:
!$SPAN_BATCH_TESTER fetch --l2 $L2_RPC --start=$ACC_L2_START --end=$ACC_L2_END --concurrent-requests=100 --out=$DB_PREFIX/singular_batches_cache | tail -10

Fetched L2 block:  3348994
Fetched L2 block:  3348992
Fetched L2 block:  3348993
Fetched L2 block:  3348997
Fetched L2 block:  3348989
Fetched L2 block:  3348995
Fetched L2 block:  3348999
Fetched L2 block:  3348998
Fetched v0 batches in range [3348000,3349000).
Fetch Config: Chain ID: 7777777.


span batch tester `merge`: Merge singular batches:

This command will analyze `start - end` number of batches, Comparing uncompressed size of below two batches:
- For `i` in range  `0` to `start - end - 1`
  - singular batch from number `start` to `start + i`, inclusive.
  - span batch from number `start` to `start + i`, inclusive.

Upper command will make a result json file which has name of the form `{start_number}_{end_number}.json`.

In [31]:
!$SPAN_BATCH_TESTER merge --start=$ACC_L2_START --end=$ACC_L2_END --l2 $L2_RPC --genesis-timestamp=$L2_GENESIS_TIMESTAMP --in=$DB_PREFIX/singular_batches_cache --out=$DB_PREFIX/merge_result | tail -10

Merged v0 batches in range [3348000,3349000).


span batch tester `format`: Compare different tx encodings for span batch.


In [21]:
!$SPAN_BATCH_TESTER format --permutation 0123465 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

Reduction Percentage: -0.307207 %
[405/408] cnt: 163, delta: -274, deltasum: -17366, originalCompressedSizeSum: 19396160
Reduction Percentage: -0.548208 %
[406/408] cnt: 164, delta: 23, deltasum: -17343, originalCompressedSizeSum: 19449603
Reduction Percentage: 0.043037 %
[407/408] cnt: 165, delta: 25, deltasum: -17318, originalCompressedSizeSum: 19507461
Reduction Percentage: 0.043209 %
[408/408] cnt: 165, delta: -286, deltasum: -17604, originalCompressedSizeSum: 19547084
Reduction Percentage: -0.721803 %
Final Reduction Percentage: -0.090059 %


In [22]:
!$SPAN_BATCH_TESTER format --permutation 0134625 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

Reduction Percentage: -0.311358 %
[405/408] cnt: 114, delta: -35, deltasum: -28940, originalCompressedSizeSum: 19396160
Reduction Percentage: -0.070027 %
[406/408] cnt: 114, delta: -24, deltasum: -28964, originalCompressedSizeSum: 19449603
Reduction Percentage: -0.044908 %
[407/408] cnt: 114, delta: -52, deltasum: -29016, originalCompressedSizeSum: 19507461
Reduction Percentage: -0.089875 %
[408/408] cnt: 114, delta: -272, deltasum: -29288, originalCompressedSizeSum: 19547084
Reduction Percentage: -0.686470 %
Final Reduction Percentage: -0.149833 %


In [23]:
!$SPAN_BATCH_TESTER format --permutation 3041265 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

Reduction Percentage: -0.290601 %
[405/408] cnt: 115, delta: -260, deltasum: -19426, originalCompressedSizeSum: 19396160
Reduction Percentage: -0.520198 %
[406/408] cnt: 116, delta: 9, deltasum: -19417, originalCompressedSizeSum: 19449603
Reduction Percentage: 0.016840 %
[407/408] cnt: 117, delta: 35, deltasum: -19382, originalCompressedSizeSum: 19507461
Reduction Percentage: 0.060493 %
[408/408] cnt: 117, delta: -281, deltasum: -19663, originalCompressedSizeSum: 19547084
Reduction Percentage: -0.709184 %
Final Reduction Percentage: -0.100593 %


In [24]:
!$SPAN_BATCH_TESTER format --permutation 0125634 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

Reduction Percentage: -0.012454 %
[405/408] cnt: 197, delta: -191, deltasum: -7234, originalCompressedSizeSum: 19396160
Reduction Percentage: -0.382145 %
[406/408] cnt: 198, delta: 15, deltasum: -7219, originalCompressedSizeSum: 19449603
Reduction Percentage: 0.028067 %
[407/408] cnt: 199, delta: 19, deltasum: -7200, originalCompressedSizeSum: 19507461
Reduction Percentage: 0.032839 %
[408/408] cnt: 199, delta: -130, deltasum: -7330, originalCompressedSizeSum: 19547084
Reduction Percentage: -0.328092 %
Final Reduction Percentage: -0.037499 %


In [25]:
!$SPAN_BATCH_TESTER format --permutation 0324165 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

Reduction Percentage: -0.307207 %
[405/408] cnt: 119, delta: -276, deltasum: -21340, originalCompressedSizeSum: 19396160
Reduction Percentage: -0.552210 %
[406/408] cnt: 120, delta: 0, deltasum: -21340, originalCompressedSizeSum: 19449603
Reduction Percentage: 0.000000 %
[407/408] cnt: 121, delta: 25, deltasum: -21315, originalCompressedSizeSum: 19507461
Reduction Percentage: 0.043209 %
[408/408] cnt: 121, delta: -294, deltasum: -21609, originalCompressedSizeSum: 19547084
Reduction Percentage: -0.741993 %
Final Reduction Percentage: -0.110548 %


In [26]:
!$SPAN_BATCH_TESTER format --permutation 3041256 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

Reduction Percentage: 0.049817 %
[405/408] cnt: 169, delta: 36, deltasum: -809, originalCompressedSizeSum: 19396160
Reduction Percentage: 0.072027 %
[406/408] cnt: 169, delta: -14, deltasum: -823, originalCompressedSizeSum: 19449603
Reduction Percentage: -0.026196 %
[407/408] cnt: 170, delta: 13, deltasum: -810, originalCompressedSizeSum: 19507461
Reduction Percentage: 0.022469 %
[408/408] cnt: 171, delta: 13, deltasum: -797, originalCompressedSizeSum: 19547084
Reduction Percentage: 0.032809 %
Final Reduction Percentage: -0.004077 %


In [27]:
!$SPAN_BATCH_TESTER format --permutation 3401256 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

Reduction Percentage: 0.045666 %
[405/408] cnt: 170, delta: 34, deltasum: -810, originalCompressedSizeSum: 19396160
Reduction Percentage: 0.068026 %
[406/408] cnt: 170, delta: -14, deltasum: -824, originalCompressedSizeSum: 19449603
Reduction Percentage: -0.026196 %
[407/408] cnt: 171, delta: 13, deltasum: -811, originalCompressedSizeSum: 19507461
Reduction Percentage: 0.022469 %
[408/408] cnt: 172, delta: 11, deltasum: -800, originalCompressedSizeSum: 19547084
Reduction Percentage: 0.027762 %
Final Reduction Percentage: -0.004093 %


In [28]:
!$SPAN_BATCH_TESTER format --permutation 6051234 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

Reduction Percentage: -0.247011 %
[405/408] cnt: 203, delta: 23, deltasum: 538, originalCompressedSizeSum: 19396160
Reduction Percentage: 0.046017 %
[406/408] cnt: 204, delta: 23, deltasum: 561, originalCompressedSizeSum: 19449603
Reduction Percentage: 0.043037 %
[407/408] cnt: 205, delta: 46, deltasum: 607, originalCompressedSizeSum: 19507461
Reduction Percentage: 0.079505 %
[408/408] cnt: 205, delta: -201, deltasum: 406, originalCompressedSizeSum: 19547084
Reduction Percentage: -0.507281 %
Final Reduction Percentage: 0.002077 %


In [29]:
!$SPAN_BATCH_TESTER format --permutation 0132456 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

Reduction Percentage: 0.010379 %
[405/408] cnt: 110, delta: 4, deltasum: -3016, originalCompressedSizeSum: 19396160
Reduction Percentage: 0.008003 %
[406/408] cnt: 110, delta: -24, deltasum: -3040, originalCompressedSizeSum: 19449603
Reduction Percentage: -0.044908 %
[407/408] cnt: 111, delta: 7, deltasum: -3033, originalCompressedSizeSum: 19507461
Reduction Percentage: 0.012099 %
[408/408] cnt: 112, delta: 4, deltasum: -3029, originalCompressedSizeSum: 19547084
Reduction Percentage: 0.010095 %
Final Reduction Percentage: -0.015496 %


In [30]:
!$SPAN_BATCH_TESTER format --permutation 0352614 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

Reduction Percentage: -0.078877 %
[405/408] cnt: 28, delta: -137, deltasum: -36505, originalCompressedSizeSum: 19396160
Reduction Percentage: -0.274104 %
[406/408] cnt: 28, delta: -99, deltasum: -36604, originalCompressedSizeSum: 19449603
Reduction Percentage: -0.185244 %
[407/408] cnt: 28, delta: -81, deltasum: -36685, originalCompressedSizeSum: 19507461
Reduction Percentage: -0.139998 %
[408/408] cnt: 28, delta: -173, deltasum: -36858, originalCompressedSizeSum: 19547084
Reduction Percentage: -0.436615 %
Final Reduction Percentage: -0.188560 %
